<a href="https://colab.research.google.com/github/vgaquino/capstone-team-29/blob/main/amazon-rekognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import argparse
import json
from datetime import datetime
import numpy as np
import zipfile
import shutil
import requests
import sys
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset
# import torchvision
# from torchvision import transforms

In [ ]:
#!pip install tqdm

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Unzip files Method 1: faster but sometimes fails.

In [ ]:
%%time
# #import all three dataset files simulatenously

# #list of zip files to be extracted
# zip_files = ['/content/drive/MyDrive/Colab Notebooks/val.zip']#, '/content/drive/MyDrive/Colab Notebooks/train.zip', '/content/drive/MyDrive/Colab Notebooks/test.zip']

# #destination folder for extracted files
# destination_folder = '/content/images'

# #create the destination folder if it doesn't exist
# os.makedirs(destination_folder, exist_ok=True)

# #loop through each zip file
# for zip_file in zip_files:
#     with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#         #extract the contents of the zip file to the destination folder
#         zip_ref.extractall(destination_folder)

# for root, dirs, files in os.walk(destination_folder):
#    for file in files:
#        shutil.move(os.path.join(root, file), os.path.join(destination_folder, file))
#    for dir in dirs:
#        shutil.rmtree(os.path.join(root, dir))


CPU times: user 24.4 s, sys: 14 s, total: 38.4 s
Wall time: 1min 3s


Unzip Files Method 2: slower but more reliable

In [ ]:
#!unzip -q -o '/content/drive/MyDrive/Colab Notebooks/train.zip' -d images
!unzip -q -o '/content/drive/MyDrive/Colab Notebooks/val.zip' -d images
#!unzip -q -o '/content/drive/MyDrive/Colab Notebooks/test.zip' -d images


In [ ]:
!rm train.zip val.zip test.zip

In [ ]:
#!mv -v images/train/* images
!mv -v images/val/* images
#!mv -v images/test/* images

In [ ]:
#extract annotations (val.json, train.json, test.json)
zip_files = ['/content/drive/MyDrive/Colab Notebooks/annotations.zip']

#destination folder for extracted files
destination_folder = '/content/'

#create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

#loop through each zip file
for zip_file in zip_files:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        #extract the contents of the zip file to the destination folder
        zip_ref.extractall(destination_folder)

In [ ]:
#Set variables
ann_path = '/content/annotations' #folder where all annotations are
images_path = '/content/images'#folder where all images from train, test, and val are
#prediction_path = 'predictions/' #folder where the predictions are stored

In [ ]:
#Path to the groundtruth labels vs. our predictions
annFile = './annotations/val.json'

#structure in the .json file:

# }
# annotations = [annotation]
# annotation = {
#     "image_id": 23410,
#     "id": 117050,
#     "caption": "A plastic rewards card lying face down on the floor."
#     "is_rejected": false,
#     "is_precanned": false,
#     "text_detected": true
# }

Method 1 (jump to 2, not working yet): Try to use the API for matching the captions with the image file names.

In [ ]:
# !git clone https://github.com/Yinan-Zhao/vizwiz-caption/

Cloning into 'vizwiz-caption'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 61 (delta 2), reused 5 (delta 2), pack-reused 51
Receiving objects: 100% (61/61), 93.65 MiB | 21.63 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
# #import API class from vizwiz
# sys.path.append('/content/vizwiz-caption/vizwiz_api')
# import vizwiz


In [ ]:
# my_instance = vizwiz.VizWiz()
# #my_instance.function_from_VizWiz()

Method 2: Try manually to append the captions to each image file name:

In [ ]:
# Read the annotations from file.json

with open(annFile, 'r') as f:
    data = json.load(f)

# Create a dictionary to store mappings of image file names to annotation captions
image_caption_mapping = {}

# Iterate through annotations and populate the mapping
for annotation in data['annotations']:
    image_id = annotation['image_id']
    caption = annotation['caption']
    file_name = f"VizWiz_train_{image_id:08}.jpg"
    image_caption_mapping[file_name] = caption

for file_name in os.listdir(images_path):
    if file_name.endswith('.jpg'):
        # Check if the file name exists in the mapping
        if file_name in image_caption_mapping:
            caption = image_caption_mapping[file_name]
            print(f"Image: {file_name}, Caption: {caption}")
        else:
            print(f"No annotation found for image: {file_name}")

# Output the matched data to a new JSON file
# with open('matched_data.json', 'w') as f:
#     json.dump(matched_data, f, indent=4)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Image: VizWiz_train_00007016.jpg, Caption: a 6 pack of different tea boxes stacked on each other
VizWiz_train_00006891.jpg
Image: VizWiz_train_00006891.jpg, Caption: A math equation for subtracting 57 from 59.
VizWiz_train_00006971.jpg
Image: VizWiz_train_00006971.jpg, Caption: A filled plastic bottle of bbq sauce on a black cloth.
VizWiz_train_00006967.jpg
Image: VizWiz_train_00006967.jpg, Caption: a pair of plaid shorts sit on a wooden table in front of a sound board
VizWiz_train_00006865.jpg
Image: VizWiz_train_00006865.jpg, Caption: The digital reading on some sort of appliance or exercise equipment.
VizWiz_train_00006842.jpg
Image: VizWiz_train_00006842.jpg, Caption: a white can with blueprint which contains some sort of condensed soup
VizWiz_train_00006782.jpg
Image: VizWiz_train_00006782.jpg, Caption: A can of Campbell's Chunky Classic Chicken Noodle Soup
VizWiz_train_00007024.jpg
Image: VizWiz_train_00007024.jpg

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Tokenize Captions

Model performance on test dataset